# <center>第7章 数据清洗和准备</center>



## 7.1 处理缺失数据

>fillna(value=None, method=None, axis=None, inplace=False),value可以是数字，字典，Series或者DataFrame

>dropna(axis=0, how='any'/'all', thresh=None, subset=None, inplace=False)

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
data=pd.Series(['aardvark','artichoke',np.nan,'avocado'])
data.isnull()

#根据轴标签(对DataFrame适用)对缺失数据进行过滤
data=pd.DataFrame({'language':{2001:'Java',2003:np.nan,2004:'vt'},
                    'hello':{2001:'bb',2002:'djsl',2003:'ssss'}})
#传入any丢弃所有含有空白值的行或列，传入all只丢弃全部是空白值的行或者列
data.dropna(axis=1,how='any'/'all',thresh=None,inplace=False)

#用fillna填充空白
#可以传入字典，但此时 axis不能指定，只能按列填充
data.fillna(value='a',axis=1,method={'backfill','bfill','pad'},inplace=True)


## 7.2 移除重复数据

### <font color="#00dddd">1.利用duplicated()去除重复数据：</font>

>pd.DataFrame.duplicated(subset,keep=['first','last','False'])

In [ ]:
import pandas as pd 
import numpy as np

data=pd.DataFrame({'k1':['one','two']*3+['two'],
                    'k2':[1,1,2,3,3,4,4]})
data.duplicated()
#还可以指定检查某一列是否重复
data.duplicated(['k1'])
#自动过滤重复项
#如果不指定列，就会默认第一列
data.drop_duplicates()
data.drop_duplicates(['k1'])

### <font color='#00dddd'>2.利用函数或者映射进行数据转换</font>

>map是Series才有的方法，作用于Series的每一个元素，传入的转换方法可以是函数、字典或者Series。applymap则是DataFrame作用于每一个元素的方法

In [ ]:
data=pd.DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                    'ounces':[4,3,12,6,7.5,8,3,5,6]})
meet_to_animal={
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}
data['food']=data['food'].str.lower()
print(data)

#转入字典和Series应该是差不多的
data['type']=data['food'].map(meet_to_animal)

#也可以传入一个能完成全部这些的函数
data['food'].map(lambda x: meet_to_animal[x.lower()])


### <font color='#00dddd'>3.替换值-'replace'</font>

>DataFrame.repalce(to_replace,value,method,inplace=False),mothod是在没有传入value时的填充方法,有'ffill','bfill','None'等

In [ ]:
import pandas as pd 
import numpy as np

In [1]:
data=pd.Series([99,-1,3,4,-999,-1000])
print(data)
data.replace(-999,np.nan)

#一次替换多个值
data.replace([99,-999],np.nan)

#要让每个值有不同的替换值,传入一个值列表
data.replace([99,-999],[np.nan,0])
#利用字典替换
dic={99:4,-1:33,-1000:np.nan}
data.replace(dic,inplace=False)

#Series/DataFrame.replace方法与Series/DataFrame.str.replace不同，后者做的是字符串的元素级替换

0      99
1      -1
2       3
3       4
4    -999
5   -1000
dtype: int64


0      4.0
1     33.0
2      3.0
3      4.0
4   -999.0
5      NaN
dtype: float64


### <font color='#00dddd'>4.重命名轴索引</font>

In [ ]:
#重命名轴索引
#轴标签也可以通过函数映射进行转换，从而得到一个新的标签
import pandas as pd
import numpy as np
data=pd.DataFrame(np.arange(12).reshape(3,4),
                index=['ohio','colorabo','new york'],
                columns=['one','two','three','four'])
#跟Series一样，轴索引也有一个map方法
transform=lambda x:x[:4].title()
#直接赋值就可以修改索引了
data.index=data.index.map(transform)
#甚至你还可以修改columns
data.columns.map(transform)
#如果要创建数据集的转换版(而不是修改原始数据),比较实用的方法是rename
#传入的str表示不变动,或者传入str.upper、str.title等等,你还可以传入字典
#rename(self,index=str,columns=?,axis=?,inplace=False)
data.rename(index=str.title,columns=str.upper)



### <font color='#00dddd'>5.离散化和面元划分-pd.cut和pd.qcut</font>

>pd.cut是按数值分组,而pd.qcut是按分位数分组

In [ ]:
import pandas as pd 
import numpy as np

In [2]:
age=np.random.randint(1,77,9)
bins=[1,33,55,77]
cat=pd.cut(age,bins=bins,right=True)
cat.codes#组别
cat.categories#分组区间
#分组统计
pd.value_counts(cat)
#还可以自己传入组名
group=['high','median','low','none']
b=pd.cut(np.random.randn(20),4,labels=group)
b.codes
b.categories
b.value_counts()

#qcut,按分位数进行分组
data=np.random.randn(1000)
cats=pd.qcut(data,q=4)
#自定义分位数
interval=[0,0.2,0.4,0.8,1]
cats=pd.qcut(data,interval)


## <font color='#00dddd'>6.检测和过滤异常值</font>

In [ ]:
import pandas as pd
import numpy as np

data=pd.DataFrame(np.random.randn(1000,4))
data.describe()
#假如你想要找出某列中绝对值超过3的值:
col=data[2]
col[np.abs(col)>3]
#找出有绝对值超过3的行
data[(np.abs(data)>3).any(1)]
#找出有绝对值超过3的列
data[(np.abs(data)>3).any(0)]


### <font color='#00dddd'>7.排列和随机取样</font>

In [ ]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.arange(20).reshape(5,4))
sample=np.random.permutation(5)
sample
df.take(sample)
df.iloc(sample)
#取样
df.sample(3)

#有放回的取样
df.sample(6,replace=True)



### <font color='#00dddd'>8.计算哑变量</font>

In [3]:
#如果DataFrame的某一列含有k个不同的值，则可以派生出一个k列矩阵
import pandas as pd
import numpy as np
df=pd.DataFrame({'key':['a','b','c','d','b','c'],'data1':np.arange(6)})
pd.get_dummies(df['key'])

#给虚拟变量加前缀
pd.get_dummies(df['key'],prefix='keys')

#统计的应用,结合get_dummies和cut之类的操作
a=np.random.randint(0,5,10)
print(a)
b=pd.get_dummies(pd.cut(a,4))
print(b)

[0 3 3 2 0 1 2 2 3 0]
   (-0.003, 0.75]  (0.75, 1.5]  (1.5, 2.25]  (2.25, 3.0]
0               1            0            0            0
1               0            0            0            1
2               0            0            0            1
3               0            0            1            0
4               1            0            0            0
5               0            1            0            0
6               0            0            1            0
7               0            0            1            0
8               0            0            0            1
9               1            0            0            0



# 7.3 字符串操作

In [ ]:
#Python内置的字符串方法
a='jfdlsjfs'
b=['jfdl','djfd','avdd','ffff']
len(a)
a.split('f')
'.'.join(a)
'/'.join(b)
#字符串定位
a.index('f')#字符串不存在就会raise Error
a.find(']')#不存在就会返回-1

### <font color='#00dddd'>1.正则表达式</font>

In [ ]:
#正则表达式,分为三个模块--模式匹配、替换以及拆分
import re

text="foo    bar\t baz  \tqux"
re.split('\s+',text)

#先把patter进行编译
#如果打算对许多字符串应用同一条正则表达式，强烈建立通过re.compile先编译，可以节省CPU时间
regex=re.compile(r'\s+')
regex.split(text)
#findall,search,match
text='''Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com'''
pattern= r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex=re.compile(pattern,flags=re.I)
regex.findall(text)
#search只返回匹配的第一个对象
a=regex.search(text)
a.start
a.end
#sub方法可以将匹配到的字符串替换为指定字符串(不改变原text)
regex.sub('REDACTED',text)
#分段
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex1=re.compile(pattern,flags=re.I)
regex1.findall(text)
#字符串替换
regex1.sub(r'Username: \1, Domain: \2, Suffix: \3',text)


### <font color='#00dddd'>2.pandas的矢量化函数</font>

In [ ]:
#pandas的矢量化字符串函数
#在进行字符串规整时，有时候会含有缺失数据
import pandas as pd
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
         'Rob': 'rob@gmail.com', 'Wes': np.nan}
data=pd.Series(data)
data.isnull()

#在通过Series.map时，存在NA就会报错
#Series有一些能够跳过NA值的方法，通过Series的str属性就可以访问这些方法比如Series.str.lower()
#map作用于单个element
data.map(str.lower)